In [68]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import math
from sklearn.metrics import mean_absolute_percentage_error
from tqdm import tqdm
import time
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow.keras.layers import Flatten

#read the data
df = pd.read_csv("Dasatest.csv",
                infer_datetime_format=True,)

#check the dtypes
# print(df.dtypes)
df['Month'] = pd.to_datetime(df.Month , format = '%m/%d/%Y')
data = df.drop(['Month'], axis=1)
data.index = df.Month
# print(df.head())
cols = data.columns



# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler = scaler.fit(dataset)
# dataset_scaled = scaler.transform(dataset)
# print(dataset_scaled)

# train = dataset_scaled[:int(0.84*(len(data)))]
# valid = dataset_scaled[int(0.84*(len(data))):]

# model = VAR(endog=train)

# data_wo_trend1 = data.iloc[:,:2000] - data.iloc[:,:2000].shift(1)
# data_wo_trend2 = data.iloc[:,2000:] - data.iloc[:,2000:].shift(2)

# frames = [data_wo_trend1,data_wo_trend2]
# data_wo_trend = pd.concat(frames,axis=1)

# data_wo_sea = data_wo_trend - data_wo_trend.shift(12)

dataset = np.array(data)
print(dataset.shape)

print(dataset)

from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

#apply adf test on the series
adf_test(dataset[:,100])

(72, 3915)
[[0.1543245  0.11970796 0.17823648 ... 0.46352725 1.64151026 0.30708925]
 [0.12290076 0.11983117 0.16549748 ... 0.57131545 1.64993684 0.45160385]
 [0.12758278 0.11297011 0.19075444 ... 0.64402526 2.11044938 0.44806313]
 ...
 [0.37375369 0.34499689 0.40026859 ... 1.20782415 1.77394841 0.81705893]
 [0.3336417  0.30050302 0.36040647 ... 1.1479989  1.64021404 0.75481252]
 [0.24217588 0.21514682 0.33592286 ... 0.76868481 1.48927726 0.54231719]]
Results of Dickey-Fuller Test:
Test Statistic                 -3.830917
p-value                         0.002606
#Lags Used                      0.000000
Number of Observations Used    71.000000
Critical Value (1%)            -3.526005
Critical Value (5%)            -2.903200
Critical Value (10%)           -2.588995
dtype: float64


In [79]:
model = VAR(dataset[:,:])
results = model.fit(3)

lag_order = results.k_ar
prediction = results.forecast(dataset[-3:,:], steps=15)

print(prediction[:,0])

#converting predictions to dataframe

# sum1 = 0 

# pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
# for j in range(0,3915):
#     for i in range(0, len(prediction)):
#        pred.iloc[i][j] = prediction[i][j]

# #check mape
# for i in cols:
#     print('rmse value for', i, 'is : ', mean_absolute_percentage_error(pred.loc[:,[i]] ,valid.loc[:,[i]]))
#     sum1 += mean_absolute_percentage_error(pred.loc[:,[i]] ,valid.loc[:,[i]])



[0.35147379 0.26660285 0.30932539 0.36398538 0.2879106  0.28832814
 0.26461797 0.32934529 0.36139373 0.38754948 0.36784502 0.34934444
 0.24063438 0.28120514 0.33225126]


In [17]:
print(sum1)

1085.747601569012


In [4]:
for k in range(6):
    model_fit = model.fit(k+1)

    prediction = model_fit.forecast(model_fit.y, steps=len(valid))

    print(train.shape)

    #converting predictions to dataframe

    sum1 = 0 
    
    pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
    for j in range(0,3915):
        for i in range(0, len(prediction)):
           pred.iloc[i][j] = prediction[i][j]
    
    #check mape
    for i in tqdm(cols):
        #print('rmse value for', i, 'is : ', mean_absolute_percentage_error(pred[i], valid[i]))
        sum1 += mean_absolute_percentage_error(pred.loc[:,[i]] ,valid.loc[:,[i]])

    print(sum1,k+1)

# model = VAR(endog=data)
# model_fit = model.fit(12)
# yhat = model_fit.forecast(model_fit.y, steps=15)
# print(yhat)

# output = pd.DataFrame(yhat)
# output.to_csv('submission_12.csv')
# print("Your submission was successfully saved!")







c:\users\dhruv\appdata\local\programs\python\python36\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


(60, 3915)


100%|█████████████████████████████████████████████████████████████████████████████| 3915/3915 [00:13<00:00, 282.82it/s]


1085.7476015690156 1


c:\users\dhruv\appdata\local\programs\python\python36\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


...

1121.6415763649857 5


c:\users\dhruv\appdata\local\programs\python\python36\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


(60, 3915)


100%|█████████████████████████████████████████████████████████████████████████████| 3915/3915 [00:11<00:00, 326.40it/s]

1102.0567142428388 6
